-----
<div> <center> ESPACIO PARA BANNER DE LA MAESTRIA </center> </div>   

-----

# Aplicación de LDA
En este Notebook usaremos datos de comentarios realizados por usuarios de Tripadvisor a restaurantes en Bogotá. El objetivo es aplicar la técnica de LDA para extraer algunos tópicos del corpus.

In [1]:
import os 
import pandas as pd

Comenzamos importando la base de datos

In [2]:
os.listdir("data")

['comentarios_tripadvisor.csv',
 'enlaces_tripadvisor.csv',
 'Scraping_tripadvisor.ipynb']

In [3]:
df = pd.read_csv("data/comentarios_tripadvisor.csv", sep = ";")
df.head()

,nombre,link,puntaje_global,n_comentarios,posicion_relativa,n_restaurantes,lat,lon,excelente,muy_bueno,...,características,usuario,relevancia_usuario,fecha_comentario,titulo_comentario,contenido_comentario,fecha_visita,puntaje,rango_de_precios,dietas_especiales
0,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Marylyn73,4 opiniones,Escribió una opinión el 6 de enero de 2021,Comida recalentada,Fui temprano para evitar aglomeraciones y me t...,enero de 2021,10.0,NaN,NaN
1,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",jairoenriquer2020,1 opinión,Escribió una opinión el 10 de marzo de 2020,Mala calidad y costo alto,Mala calidad de la carne no es la mamona tradi...,marzo de 2020,10.0,NaN,NaN
2,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",HectorLatorre,55 opiniones,Escribió una opinión el 13 de febrero de 2020,Maravilloso sitio de comida llanera,Este es un restaurante bastante típico y muy b...,enero de 2020,40.0,NaN,NaN
3,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",Ingriofercas,1 opinión,Escribió una opinión el 13 de enero de 2020,Buen Restaurante Tipico!!!,"buen servicio, eramos mas o menos 20 personas ...",diciembre de 2019,50.0,NaN,NaN
4,Asadero Cimarron del Llano,/Restaurant_Review-g294074-d10003846-Reviews-A...,"4,0",17.0,83.0,127.0,4.666695,-74.113075,9.0,3.0,...,"Comida para llevar, Servicio de mesa, Reservas...",profet2016,429 opiniones,Escribió una opinión el 12 de octubre de 2019,Un recuerdo de la cultura llanera,"Muy buen ambiente, buena comida y muy buena mú...",octubre de 2019,50.0,NaN,NaN


En total tenemos más de 130 mil comentarios (los cuales representan cada una de las filas del dataframe) y 25 variables que describen el restaurante y el comentario.

In [4]:
df.shape 

(132728, 25)

Primero vamos a quedarnos solo con las columnas que nos interesan: `titulo_comentario` y `contenido_comentario`. Luego las unimos.

In [5]:
# Vamos a reducir el tamaño de nuestra muestra para facilitar le computo de
# los procesos
df = df.sample(10000, random_state = 666).reset_index(drop = True)

In [6]:
comentarios = df["titulo_comentario"] + " " + df["contenido_comentario"]

Procedemos a preprocesar el texto

In [7]:
import unidecode
import re

# Convertimos la columna en texto
comentarios = comentarios.astype(str)

# Quitamos tildes
comentarios = comentarios.apply(lambda x: unidecode.unidecode(x))

# Quitamos comas, guiones y otros caracteres especiales o signos de puntuación
comentarios = comentarios.apply(lambda x:
    re.sub('[^A-Za-z0-9 ]+', ' ', x))

# Ponemos todo el texto en minúscula 
comentarios = comentarios.str.lower()

# Dejamos todos los espacios sencillos
comentarios = comentarios.apply(lambda x: 
    re.sub('\s+', ' ', x))

# Vamos a eliminar todos los números
comentarios = comentarios.apply(lambda x: re.sub("\d+", "", x))
comentarios = comentarios.apply(lambda x: re.sub('\s+', ' ', x))
comentarios = comentarios.str.strip()

In [8]:
# Eliminamos stopwords
from nltk.corpus import stopwords
stopwords = set(stopwords.words("spanish"))

# Creamos un diccionario de stopwords en español
stopwords = [unidecode.unidecode(i) for i in stopwords]
stopwords = set(stopwords)

# Creamos una función que elimine las palabras presentes en un diccionario
def eliminar_stopwords(texto, diccionario):
    texto = [tok for tok in texto.split(" ") if tok not in diccionario]
    return(texto)

# Aplicamos la función para eliminar los stopwords
comentarios = comentarios.apply(lambda x: 
    eliminar_stopwords(x, stopwords))

In [9]:
# Tokenizamos el texto
import spacy
nlp = spacy.load("es_core_news_sm")

# Esto puede tardar un poco
comentarios = comentarios.apply(lambda x: nlp(" ".join(x)))

In [10]:
# Lemmatizamos el texto
comentarios = comentarios.apply(lambda x: [unidecode.unidecode(tok.lemma_) for tok in x])

In [27]:
# Para aplicar LDA necesitamos construir un diccionario
import gensim.corpora as corpora

# A cada palabra se le asigna un id
diccionario = corpora.Dictionary(comentarios)

diccionario.most_common()[0:10]

[('buen', 6624),
 ('comida', 5215),
 ('excelente', 3963),
 ('lugar', 3786),
 ('servicio', 3631),
 ('restaurante', 3281),
 ('atencion', 2701),
 ('plato', 2390),
 ('mejor', 2166),
 ('ambiente', 2152)]

In [31]:
# Creamos un corpus
textos = comentarios.copy()

In [32]:
# Frequencias 
corpus = [diccionario.doc2bow(texto) for texto in textos]

In [59]:
# Aplicamos LDA
from gensim.models.ldamulticore import LdaMulticore
from pprint import pprint

# Debemos escoger el número de tópicos antes de correr el modelo
n_topicos = 3
# Modelo LDA
lda_model = LdaMulticore(corpus = corpus,
    id2word = diccionario,
    num_topics = n_topicos)
# Mostramos las palabras dentro de los 10 tópicos
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.022*"comida" + 0.020*"buen" + 0.014*"restaurante" + 0.013*"excelente" + '
  '0.013*"servicio" + 0.012*"lugar" + 0.011*"bien" + 0.011*"mejor" + '
  '0.010*"plato" + 0.009*"ambiente"'),
 (1,
  '0.038*"buen" + 0.027*"comida" + 0.016*"servicio" + 0.016*"excelente" + '
  '0.014*"lugar" + 0.014*"restaurante" + 0.013*"atencion" + 0.012*"precio" + '
  '0.010*"delicioso" + 0.010*"plato"'),
 (2,
  '0.025*"buen" + 0.021*"lugar" + 0.020*"excelente" + 0.016*"comida" + '
  '0.016*"servicio" + 0.013*"restaurante" + 0.012*"atencion" + '
  '0.010*"ambiente" + 0.010*"plato" + 0.009*"mejor"')]


In [60]:
# Visualizamos los resultados
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()
LDA_visualization = gensimvis.prepare(lda_model, corpus, diccionario)

C:\Users\User\anaconda3\envs\MIAD\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [61]:
LDA_visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.011574  0.014165       1        1  37.603392
0     -0.008158 -0.015900       2        1  31.732748
1      0.019732  0.001735       3        1  30.663860, topic_info=          Term         Freq        Total Category  logprob  loglift
25        buen  6210.000000  6210.000000  Default  30.0000  30.0000
233       bien  1487.000000  1487.000000  Default  29.0000  29.0000
93   delicioso  1288.000000  1288.000000  Default  28.0000  28.0000
1       comida  4876.000000  4876.000000  Default  27.0000  27.0000
31      precio  1566.000000  1566.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
8        mejor   472.025623  2029.074267   Topic3  -4.9912  -0.2762
241      hacer   309.472671  1061.130063   Topic3  -5.4134  -0.0501
57      comido   352.056223  1353.913966   Topic3  -5.2845  -0.1649
183      sitio   289.395392  1277.793255   Topic3  -5.4805  -0.3030
176    calidad   267.253120   991.246513   Topic3  -5.5601  -0.1287

[279 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
17         1  0.294132   acogedor
17         2  0.438787   acogedor
17         3  0.270023   acogedor
33         1  0.343934  agradable
33         2  0.465016  agradable
...      ...       ...        ...
16         1  0.577300       vivo
16         2  0.267268       vivo
16         3  0.153234       vivo
7459       1  0.790890     walter
11484      2  0.718123      wraps

[427 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

Note que puede seleccionar manualmente cada tema para ver sus términos más frecuentes y/o "relevantes", utilizando diferentes valores del parámetro $\lambda$. Esto puede ayudar cuando intenta asignar un nombre interpretable por humanos o un "significado" a cada tema.

Los valores de lambda ($\lambda$) que están muy cerca de cero mostrarán términos que son más específicos para un tema elegido. Lo que significa que verá términos que son "importantes" para ese tema específico pero no necesariamente "importantes" para todo el corpus.

Los valores de lambda que están muy cerca de uno mostrarán aquellos términos que tienen la relación más alta entre la frecuencia de los términos para ese tema específico y la frecuencia general de los términos del corpus.

In [62]:
# Guardamos la visualización como un html. 
# Es mucho más sencillo interactuar con la gráfica desde el archivo que 
# desde el notebook
pyLDAvis.save_html(LDA_visualization, 'visualizacion_LDA.html')